In [1]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr

In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Valid Key exists")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Valid Key exists")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Valid Key exists")
else:
    print("Google API Key not set")

OpenAI API Valid Key exists
Anthropic API Valid Key exists
Google API Valid Key exists


In [3]:
#Check tghe connections with LLM

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

In [4]:
system_message = "You are a helpful assistant"

In [5]:
def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

In [6]:
# Let's check the traning date cut off time

message_gpt("What is today's date?")

"Today's date is October 5, 2023."

In [7]:
def makeupper(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [8]:
makeupper("hello")

Shout has been called with input hello


'HELLO'

In [9]:
gr.Interface(fn=makeupper, inputs="textbox", outputs="textbox").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [10]:
# Checking Share equals true condition
gr.Interface(fn=makeupper, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


In [11]:
#Auto Launch in the Browser 
gr.Interface(fn=makeupper, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [12]:
# Define this variable and then pass js=force_dark_mode to enable Dark mode

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=makeupper, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [43]:
view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response from OpenAI Model:", lines=8)],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


In [14]:
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [45]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response from OpenAI Model:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


In [16]:
from anthropic import Anthropic

def stream_claude(prompt):
    client = Anthropic()  # Initialize the client
    
    result = client.messages.create(  # Changed from .stream to .create
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,  # Make sure system_message is defined
        messages=[
            {"role": "user", "content": prompt},
        ],
        stream=True  # Add this parameter to enable streaming
    )
    
    response = ""
    for chunk in result:  # Simplified streaming iteration
        if chunk.type == "content_block_delta":  # Check for content updates
            text = chunk.delta.text
            response += text
            yield response

In [49]:
view = gr.Interface(
    fn=stream_claude,
    inputs=[gr.Textbox(label="Your question to Anthropic LLM:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


In [18]:
import google.generativeai as genai

def stream_gemini(prompt):
    # Initialize the Gemini client
    genai.configure(api_key=google_api_key)
    
    # Initialize the model
    model = genai.GenerativeModel('gemini-1.5-pro')
    
    # Create the chat
    chat = model.start_chat(
        history=[],
    )
    
    # Generate streaming response
    response = ""
    result = chat.send_message(
        prompt,
        stream=True,
        generation_config=genai.types.GenerationConfig(
            temperature=0.7,
            max_output_tokens=1000,
        )
    )
    
    # Stream the response
    for chunk in result:
        # Extract text from the chunk
        text = chunk.text
        response += text
        yield response

In [51]:
view = gr.Interface(
    fn=stream_gemini,
    inputs=[gr.Textbox(label="Your question to Gemini LLM:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.
